starting out by replicating [this notebook](https://www.kaggle.com/sbongo/for-beginners-tackling-toxic-using-keras) for beginners, to get a feel for Keras.

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [5]:
test.isnull().any()

id              False
comment_text    False
dtype: bool

Theres no nulls, so we can be lazy with preprocessing

In [6]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [7]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [8]:
max_len = 200
X_tr = pad_sequences(list_tokenized_train, maxlen=max_len)
X_te = pad_sequences(list_tokenized_test, maxlen=max_len)

In [9]:
inp = Input(shape=(max_len, ))

In [10]:
embed_size = 128
x = Embedding(max_features, embed_size)(inp)

In [11]:
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)

In [12]:
x = GlobalMaxPool1D()(x)

In [13]:
x = Dropout(0.1)(x)

In [14]:
x = Dense(50, activation="relu")(x)

In [15]:
x = Dropout(0.1)(x)

In [16]:
x = Dense(6, activation="sigmoid")(x)

In [17]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 2
model.fit(X_tr,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
   928/143613 [..............................] - ETA: 46:11 - loss: 0.4991 - acc: 0.8949